In [ ]:
import os
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import kagglehub
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


class OlistSatisfactionModel:
    """
    Pipeline para predecir satisfacción del cliente en Olist.

    Enfoque :
    - y (target) se construye explícitamente con make_target()
    - X (features) se construye explícitamente con build_features()      
    Ventajas:
    - Evita leakage por diseño (no hay drops largos: simplemente no incluimos lo prohibido).
    - Es fácil ver qué entra al modelo (X) y qué se predice (y).
    - Si el dataset trae columnas nuevas, NO entran automáticamente al modelo.
    """

    def __init__(self, n_estimators: int = 200, random_state: int = 42, n_jobs: int = -1) -> None:
        """
        Parameters
        ----------
        n_estimators : int
            Número de árboles para RandomForest.
        random_state : int
            Semilla de reproducibilidad.
        n_jobs : int
            Paralelización (-1 usa todos los cores).
        """
        self.model: Optional[RandomForestClassifier] = None

        # Metadata necesaria para inferencia consistente
        self.model_columns: Optional[pd.Index] = None
        self.numeric_means: Optional[pd.Series] = None
        self.freight_median_: Optional[float] = None

        self.n_estimators = n_estimators
        self.random_state = random_state
        self.n_jobs = n_jobs

    # ---------------------------------------------------------------------
    # 1) Carga de datos 
    # ---------------------------------------------------------------------
    def load_raw_data(self) -> pd.DataFrame:
        """
        Descarga el dataset de Olist vía kagglehub y realiza el merge principal
        para obtener un DataFrame unificado.

        Returns
        -------
        pd.DataFrame
            DataFrame unificado (a nivel item dentro de order).
        """
        #print("Descargando/Cargando datos...")
        #path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

        #data: Dict[str, pd.DataFrame] = {}
        #for file in os.listdir(path):
        #    if file.endswith(".csv"):
        #        key = file.split(".")[0]
        #        data[key] = pd.read_csv(os.path.join(path, file))

        #df = (
        #    data["olist_orders_dataset"]
        #    .merge(data["olist_order_reviews_dataset"], on="order_id", how="left")
        #    .merge(data["olist_order_payments_dataset"], on="order_id", how="left")
        #    .merge(data["olist_customers_dataset"], on="customer_id", how="left")
        #    .merge(data["olist_order_items_dataset"], on="order_id", how="left")
        #    .merge(data["olist_products_dataset"], on="product_id", how="left")
        #    .merge(data["olist_sellers_dataset"], on="seller_id", how="left")
        #)
        dataset="dsunified.csv"
        df= pd.read_csv(f'data/{dataset}')
        return df

    # ---------------------------------------------------------------------
    # 2) Target explícito (Y)
    # ---------------------------------------------------------------------
    def make_target(self, df: pd.DataFrame) -> pd.Series:
        """
        Construye target (y) desde review_score.

        Definición:
        - y = 1 si review_score >= 4
        - y = 0 si review_score < 4

        Parameters
        ----------
        df : pd.DataFrame
            Debe contener la columna 'review_score' (solo training).

        Returns
        -------
        pd.Series
            Vector y de 0/1.
        """
        if "review_score" not in df.columns:
            raise KeyError("Training requiere 'review_score' para construir el target.")
        return (df["review_score"] >= 4).astype(int)

    # ---------------------------------------------------------------------
    # 3) features (X) explícito (whitelist)
    # ---------------------------------------------------------------------
    def build_features(self, df: pd.DataFrame, is_training: bool) -> pd.DataFrame:
        """
        Construye features (X) de forma explícita (whitelist).
        Solo incluye:
        - features temporales (deltas y componentes)
        - features de precio/flete/revenue
        - volumen de producto
        - algunas categóricas de baja cardinalidad (state, payment_type, product_category_name)

        IMPORTANT:
        - No usa ni incluye review_score en X (evita leakage).
        - Aprende un umbral fijo (freight_median_) en training y lo reutiliza en inference.

        Parameters
        ----------
        df : pd.DataFrame
            DataFrame crudo (merge) o batch nuevo con columnas equivalentes.
        is_training : bool
            True en entrenamiento, False en inferencia.

        Returns
        -------
        pd.DataFrame
            DataFrame X con features listas para preprocesamiento.
        """
        df = df.copy()

        # --- Parseo de fechas mínimas necesarias ---
        if "order_purchase_timestamp" not in df.columns:
            raise KeyError("Falta 'order_purchase_timestamp' (requerida).")

        df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"], errors="coerce")

        date_cols = [
            "order_approved_at",
            "order_delivered_carrier_date",
            "order_delivered_customer_date",
            "order_estimated_delivery_date",
            "shipping_limit_date",
        ]
        for c in date_cols:
            if c in df.columns:
                df[c] = pd.to_datetime(df[c], errors="coerce")

        base = df["order_purchase_timestamp"]

        # --- X explícito (solo lo que queremos que exista) ---
        X = pd.DataFrame(index=df.index)

        # 1) Deltas (days)
        def delta_days(col: str) -> pd.Series:
            if col not in df.columns:
                return pd.Series(np.nan, index=df.index)
            return (df[col] - base).dt.days

        X["delta_approved"] = delta_days("order_approved_at")
        X["delta_estimated_delivery"] = delta_days("order_estimated_delivery_date")
        X["delta_shipping_limit"] = delta_days("shipping_limit_date")
        X["delta_delivered_customer"] = delta_days("order_delivered_customer_date")
        X["delta_delivered_carrier"] = delta_days("order_delivered_carrier_date")

        # 2) Componentes fecha
        X["purchase_year"] = base.dt.year
        X["purchase_month"] = base.dt.month
        X["purchase_day"] = base.dt.day

        # 3) Volumen producto (si existen dimensiones)
        dims = {"product_length_cm", "product_height_cm", "product_width_cm"}
        if dims.issubset(df.columns):
            X["product_cubic_volume"] = (
                df["product_length_cm"] * df["product_height_cm"] * df["product_width_cm"]
            )
        else:
            X["product_cubic_volume"] = np.nan

        # 4) Precio/Flete/Revenue (si existen columnas)
        required_pf = {"price", "freight_value"}
        if required_pf.issubset(df.columns):
            # Evita división por 0: si price=0 -> NaN
            price = df["price"].replace(0, np.nan)

            X["freight_percentage"] = df["freight_value"] / price
            X["net_revenue"] = df["price"] - df["freight_value"]
            X["revenue_per_order"] = df["price"] + df["freight_value"]
        else:
            X["freight_percentage"] = np.nan
            X["net_revenue"] = np.nan
            X["revenue_per_order"] = np.nan

        # 5) Flag de freight alto con umbral fijo aprendido
        if is_training:
            self.freight_median_ = float(pd.Series(X["freight_percentage"]).median(skipna=True))

        if self.freight_median_ is None:
            raise RuntimeError("freight_median_ no inicializado. Entrena el modelo primero.")

        X["is_high_freight"] = (X["freight_percentage"] > self.freight_median_).astype(int)

        # 6) Categóricas (baja cardinalidad y razonables en prod)
        # Nota: evitar customer_city/seller_city por cardinalidad.
        cat_allow = [
            "customer_state",
            "seller_state",
            "payment_type",
            "product_category_name",
        ]
        for col in cat_allow:
            X[col] = df[col] if col in df.columns else np.nan

        return X

    # ---------------------------------------------------------------------
    # 4) Preprocess: imputación + one-hot + alineación
    # ---------------------------------------------------------------------
    def preprocess(self, X: pd.DataFrame, is_training: bool) -> pd.DataFrame:
        """
        Preprocesa X para el modelo:
        - Imputa numéricos con medias aprendidas en training.
        - En inferencia, rellena categóricos faltantes con 'Unknown'.
        - One-hot encoding con pd.get_dummies.
        - En inferencia, reindexa para coincidir con columnas vistas en training.

        Parameters
        ----------
        X : pd.DataFrame
            Features construidas por build_features().
        is_training : bool
            True para aprender estadísticas y columnas; False para aplicar.

        Returns
        -------
        pd.DataFrame
            Matriz final model-ready.
        """
        X = X.copy()

        # 1) Medias numéricas (training) + imputación (train/infer)
        num_cols = X.select_dtypes(include=["number", "bool"]).columns
        if is_training:
            self.numeric_means = X[num_cols].mean(numeric_only=True)

        if self.numeric_means is None:
            raise RuntimeError("numeric_means no inicializado. Entrena el modelo primero.")

        cols_to_fill = [c for c in self.numeric_means.index if c in X.columns]
        if cols_to_fill:
            X[cols_to_fill] = X[cols_to_fill].fillna(self.numeric_means[cols_to_fill])

        #2) Categóricas: Imputación consistente "Unknown"
        cat_cols = X.select_dtypes(include=["object", "string"]).columns
        if len(cat_cols) > 0:
            X[cat_cols] = X[cat_cols].fillna("Unknown")

        # Limpieza final de filas en training (solo si quedan nulos rebeldes)
        if is_training:
            # En entrenamiento: Queremos datos puros.
            # Si algo falló en la imputación y sigue siendo nulo, lo borramos.
            # No queremos que el modelo aprenda de datos corruptos.
            X = X.dropna()
        else:
            # En inferencia: PROHIBIDO BORRAR FILAS.
            # Si queda algún nulo rebelde (ej. una columna numérica que era 
            # 100% nula en el training y no tiene media), lo rellenamos con 0.
            # Esto evita que Sklearn falle (ValueError) y garantiza una respuesta.
            X = X.fillna(0)
            
        # 3) One-hot
        X_enc = pd.get_dummies(X)

        if is_training:
            self.model_columns = X_enc.columns

        if self.model_columns is None:
            raise RuntimeError("model_columns no inicializado. Entrena el modelo primero.")

        if not is_training:
            X_enc = X_enc.reindex(columns=self.model_columns, fill_value=0)

        return X_enc

    # ---------------------------------------------------------------------
    # 5) Train / Predict
    # ---------------------------------------------------------------------
    def train(self, test_size: float = 0.2) -> None:
        """
        Entrena RandomForest usando separación explícita X/y.

        Parameters
        ----------
        test_size : float
            Proporción de test.
        """
        df_raw = self.load_raw_data()

        # y explícito
        y = self.make_target(df_raw)

        # X explícito
        X = self.build_features(df_raw, is_training=True)

        # Transformación final
        X_final = self.preprocess(X, is_training=True)

        # Alinear y con X_final (por el dropna del training en preprocess)
        # Asegura mismos índices
        y = y.loc[X_final.index]

        X_train, X_test, y_train, y_test = train_test_split(
            X_final, y, test_size=test_size, random_state=self.random_state, stratify=y
        )

        print("Entrenando Random Forest...")
        self.model = RandomForestClassifier(
            n_estimators=self.n_estimators,
            random_state=self.random_state,
            n_jobs=self.n_jobs,
        )
        self.model.fit(X_train, y_train)

        y_pred = self.model.predict(X_test)
        print("Reporte de Clasificación:")
        print(classification_report(y_test, y_pred))
        print("Modelo entrenado exitosamente.")

    def predict(self, new_data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Predice clase y probabilidad para datos nuevos.

        Parameters
        ----------
        new_data : pd.DataFrame
            DataFrame crudo. En producción real no incluirá review_score.

        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            (preds, probs)
        """
        if self.model is None:
            raise RuntimeError("El modelo no ha sido entrenado. Ejecuta .train() primero.")

        X = self.build_features(new_data, is_training=False)
        X_final = self.preprocess(X, is_training=False)

        preds = self.model.predict(X_final)
        probs = self.model.predict_proba(X_final)[:, 1]
        return preds, probs

    # ---------------------------------------------------------------------
    # 6) Save / Load Model
    # ---------------------------------------------------------------------
    def save(self, path: str) -> None:
        """
        Guarda el modelo y metadata necesaria para inferencia.

        Parameters
        ----------
        path : str
            Ruta del archivo joblib.
        """
        payload = {
            "model": self.model,
            "model_columns": self.model_columns,
            "numeric_means": self.numeric_means,
            "freight_median_": self.freight_median_,
            "n_estimators": self.n_estimators,
            "random_state": self.random_state,
            "n_jobs": self.n_jobs,
        }
        joblib.dump(payload, path)

    @classmethod
    def load(cls, path: str) -> "OlistSatisfactionModel":
        """
        Carga un pipeline guardado.

        Parameters
        ----------
        path : str
            Ruta al joblib guardado.

        Returns
        -------
        OlistSatisfactionModel
            Instancia lista para inferencia.
        """
        payload = joblib.load(path)
        obj = cls(
            n_estimators=payload.get("n_estimators", 200),
            random_state=payload.get("random_state", 42),
            n_jobs=payload.get("n_jobs", -1),
        )
        obj.model = payload["model"]
        obj.model_columns = payload["model_columns"]
        obj.numeric_means = payload["numeric_means"]
        obj.freight_median_ = payload["freight_median_"]
        return obj


if __name__ == "__main__":
    pipeline = OlistSatisfactionModel(n_estimators=200)

    # 1) Entrenar
    pipeline.train(test_size=0.2)

    # 2) Simular predicción con una muestra
    print("\n--- Simulando Predicción en Producción ---")
    df_raw = pipeline.load_raw_data().sample(5, random_state=42)

    # En producción real review_score no existe; si aparece lo removemos (no afecta X igualmente)
    if "review_score" in df_raw.columns:
        df_raw = df_raw.drop(columns=["review_score"])

    preds, probs = pipeline.predict(df_raw)
    results = pd.DataFrame({
        "Predicción (1=Satisfecho 0=No satisfecho)": preds,
        "Probabilidad": probs
    })
    print(results)

    # 3) Guardar / cargar
    #os.makedirs("output", exist_ok=True)
    #model_path = "output/olist_rf_claro.joblib"
    #pipeline.save(model_path)

    #loaded = OlistSatisfactionModel.load(model_path)
    #preds2, probs2 = loaded.predict(df_raw)
    #print("\nOK tras cargar:", preds2)


Entrenando Random Forest...
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.88      0.44      0.59      5756
           1       0.84      0.98      0.91     17710

    accuracy                           0.85     23466
   macro avg       0.86      0.71      0.75     23466
weighted avg       0.85      0.85      0.83     23466

Modelo entrenado exitosamente.

--- Simulando Predicción en Producción ---
   Predicción (1=Satisfecho 0=No satisfecho)  Probabilidad
0                                          0         0.100
1                                          0         0.160
2                                          0         0.135
3                                          1         0.945
4                                          0         0.225
